## Run de novo motif analysis on different subsets of TSS.
## 1. Extract the TSS of the regions of interest from the bed file using the Tissues in the meta file. 
### Subsets:
## Unique:
### 1. TSS unique to a tissue. 
### 2. TSS in a tissue with more than 2SD more than the median. (Mean absolute deviance)
## Houekeeping
### 3. TSS in housekeeping genes (max/median < 2) 
    

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../../parameters/params.yaml'
import yaml
import sys
import os
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

#data_folder = doc['data_folder']
#tissues = doc['tissues'].split(',')
#sys.path.append(doc['pipeline_path'])
ref_fa = doc['GENOME_FA']
#anno_gff=doc['annotation']
#mRNA_peak_file = doc["mRNA_peak_file"]
#tss_annotation = doc['tss_annotation']


import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from os.path import join
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
#from Homer import *

homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


/data2/resources/software/homer/v4.11_10242019/bin//homer


In [2]:
import click

In [3]:
from tss.utils import wrap_findMotifs
from tss.data.data_io import read_bed_file
from tss.data.data_io import write_bed_file
from tss.utils import wrap_findMotifs
from tss.data import data_io

params {'PREFIX': 'ncbi_pice_103', 'ROOT_DIR': '/data/isshamie/TSS/', 'DATA_DIR': '/data/isshamie/TSS/NCBI_PICR_103_Processed/', 'DATA_PROCESSED_DIR': '/data/isshamie/TSS/TSS/data/processed/', 'SUPPLEMENTAL_DIR': '/data/isshamie/TSS/Analysis/supplemental/', 'PIPELINE': '/data/isshamie/TSS/tss', 'GENOME_DIR': '/data/isshamie/genome', 'GENOME_FA': '/data/isshamie/genome/picr.fa', 'GENOME_GFF3': '/data/isshamie/genome/updated_final_sort.gff3', 'GENOME_GTF': '/data/isshamie/genome/updated_final_sort.gtf', 'TISSUES': 'BMDM1hKLA,BMDMwt,Brain,FemaleReproductive,Heart,Intestine,', 'META_FILE': '/data/isshamie/TSS/TSS/parameters/raw_metaSamples.tsv'}


In [4]:
os.chdir(doc["results"])
genome="alt"
outdir= join("de_novo_motifs",genome,'promoters')

if not os.path.exists(outdir):
    os.makedirs(outdir)

## Outdir:

In [5]:
unique_gene_peaks_dir = "unique_genes/"

unique_dir = "/data/isshamie/TSS/Analysis/TSS_code/figures/alt/barplots_eTSS/bmdm/" #gene_tissues_expressedLiver.uniqueGenes.txt"")


## Define unique tissues

In [6]:
# types = {'A': {'Type': 'Unique', 'Method':'Threshold','Metric': 'Peak Significance', 'Value':True,
#               'Bg':["other", "NotInTissue"]},
#          'B':{'Type': 'Unique', 'Method':'Median Deviation', 'Value':2}, 
#          'C':{'Type':'Housekeeping','Method':'Median Max','Ratio': 2},
#          'D': {'Type':'All', 'Method':'All'}}
# types

In [7]:
os.chdir(doc["results"])
TSS1_bed = read_bed_file('alt/eTSS/TSS.exp.bed')
TSS1_meta = pd.read_csv('alt/eTSS/TSS.exp.meta.tsv',sep="\t",index_col=0)
unique_genes_dir = "unique_genes"
silent_bed_f = 'alt/eTSS/TSS1.ref.bed'


## First, use all

## Run over entire Homer bed

## Default settings

## Use silenced genes as control

# Use unique genes
## Need to create a background. One is peaks which did not have that tissue in it.

In [8]:
background_bed_out = join(outdir, 'bg_files')
if not os.path.exists(background_bed_out):
    os.mkdir(background_bed_out)

In [10]:
for tis_bed in glob.glob(join(unique_gene_peaks_dir,"*uniqueGenes.bed")):
    #print(tis_bed)
    tis = basename(tis_bed).split('.uniqueGenes.bed')[0]
    tis_out = join(outdir, tis + '.uniqueGenes')
    #print(tis_out)
#     if not os.path.exists(tis_out):
#         os.mkdir(tis_out)

    bg_meta = TSS1_meta[~(TSS1_meta["Tissues"].str.contains(tis))]
    bg_bed = TSS1_bed.loc[bg_meta.index]
    out_bed = join(background_bed_out,tis+'.notPresent.bed')
    if tis == "BMDM":
        continue
    else:
        print(tis_out)
        print(out_bed)        
        write_bed_file(bg_bed,out_bed)
        wrap_findMotifs.run_findMotifs(tis_bed, tis_out, ref_fa, args=None,  bg=out_bed)

de_novo_motifs/alt/promoters/Spleen.uniqueGenes
de_novo_motifs/alt/promoters/bg_files/Spleen.notPresent.bed
nohup findMotifsGenome.pl unique_genes/Spleen.uniqueGenes.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de_novo_motifs/alt/promoters/Spleen.uniqueGenes -size 200 -len 6,8,10,12      -bg de_novo_motifs/alt/promoters/bg_files/Spleen.notPresent.bed  > de_novo_motifs/alt/promoters/Spleen.uniqueGenes.log
de_novo_motifs/alt/promoters/Liver.uniqueGenes
de_novo_motifs/alt/promoters/bg_files/Liver.notPresent.bed
nohup findMotifsGenome.pl unique_genes/Liver.uniqueGenes.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de_novo_motifs/alt/promoters/Liver.uniqueGenes -size 200 -len 6,8,10,12      -bg de_novo_motifs/alt/promoters/bg_files/Liver.notPresent.bed  > de_novo_motifs/alt/promoters/Liver.uniqueGenes.log
de_novo_motifs/alt/promoters/FemaleReproductive.uniqueGenes
de_novo_motifs/alt/promoters/bg_files/FemaleReproductive.not

## Run with just GC matched

In [11]:
for tis_bed in glob.glob(join(unique_gene_peaks_dir,"*uniqueGenes.bed")):
    print(tis_bed)
    tis = basename(tis_bed).split('.uniqueGenes.bed')[0]
    tis_out = join(outdir, tis + '.uniqueGenes.noBg')
    print(tis_out)

    wrap_findMotifs.run_findMotifs(tis_bed, tis_out, ref_fa, args=None,  bg=None,num_cores=4)

unique_genes/Spleen.uniqueGenes.bed
de_novo_motifs/alt/promoters/Spleen.uniqueGenes.noBg
nohup findMotifsGenome.pl unique_genes/Spleen.uniqueGenes.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de_novo_motifs/alt/promoters/Spleen.uniqueGenes.noBg -size 200 -len 6,8,10,12 -p 4  > de_novo_motifs/alt/promoters/Spleen.uniqueGenes.noBg.log
unique_genes/Liver.uniqueGenes.bed
de_novo_motifs/alt/promoters/Liver.uniqueGenes.noBg
nohup findMotifsGenome.pl unique_genes/Liver.uniqueGenes.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de_novo_motifs/alt/promoters/Liver.uniqueGenes.noBg -size 200 -len 6,8,10,12 -p 4  > de_novo_motifs/alt/promoters/Liver.uniqueGenes.noBg.log
unique_genes/FemaleReproductive.uniqueGenes.bed
de_novo_motifs/alt/promoters/FemaleReproductive.uniqueGenes.noBg
nohup findMotifsGenome.pl unique_genes/FemaleReproductive.uniqueGenes.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna de